# Creación de los corpus para pruebas del corrector ortográfico

In [1]:
from collections import Counter
import pandas as pd
import numpy as np
import re

language = "yi"

Inicialización del dataframe, eliminación de espacios blancos multiples, eliminación de espacios en blanco al inicio y final de las oraciones; y cambiar valores del dataframe vacíos a NaN

In [2]:
df = pd.read_excel("trabajo.xlsx", "General", skiprows=9, usecols="B:F", names=["number", "word" , "sentence", "sentence_with_errors", "error_types"])

df = df.replace({"\s+": " "}, regex=True)
df = df.apply(lambda row: row.str.strip() if row.dtype == "object" else row)
df["sentence"] = df["sentence"].str.lower()
df["sentence_with_errors"] = df["sentence_with_errors"].str.lower()
df["error_types"] = df["error_types"].str.replace(" ", "")
df["error_types"] = df["error_types"].str.upper()
df = df.replace(r"^\s*$", np.nan, regex=True)

In [3]:
print("Número de oraciones: {}".format(df.shape[0]))

Número de oraciones: 2200


Remover las filas que no tengan una oración y que no tengan ningún tipo de error etiquetado

In [4]:
df = df[df["sentence"].notna()]
df = df[df["sentence_with_errors"].notna()]
df = df[df["error_types"].notna()]

In [5]:
print("Número de oraciones: {}".format(df.shape[0]))

Número de oraciones: 2082


Identificar las filas que tienen un error en el formato de la colummna "error types"

In [6]:
error_numbers = []
possible_error_types = ["FON", "GEN", "TIE", "NUM", "PUN", "ACE", "SIN", "SEM"]

for _, row in df.iterrows():
    error_types = row["error_types"]
    error_types_list = error_types.split(',')
    for error_type in error_types_list:
        if error_type not in possible_error_types:
            error_numbers.append(row["number"])
            break

In [7]:
print("{} oraciones con errores de formato en la columna \"error types\": {}".format(len(error_numbers), error_numbers))

0 oraciones con errores de formato en la columna "error types": []


Remover las filas en donde no se pudo corregir manualmente el error de formato en la columna "error types"

In [8]:
df = df[~df["number"].isin(error_numbers)]

In [9]:
print("Número de oraciones: {}".format(df.shape[0]))

Número de oraciones: 2082


Agregar la columna "error_types_count" al dataframe que indique el número de "error_types"

In [10]:
df["error_types_count"] = df.apply(lambda row: len(row["error_types"].split(',')), axis=1)

Identificar los caracteres únicos de las oraciones para propósitos de limpieza manual del excel

In [11]:
chars = sorted(list(set((df["sentence"] + df["sentence_with_errors"]).sum())))
print("Lista de caracteres (ambos conjuntos de oraciones combinadas): {}".format(chars))

Lista de caracteres (ambos conjuntos de oraciones combinadas): [' ', ',', '.', 'a', 'c', 'e', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'ñ']


Agregar las columnas "words" y "words_with_errors" para identificar las palabras en las oraciones que tienen errores ortográficos, también identificaremos las filas en donde las oraciones no coinciden en número de palabras

In [12]:
df["words"] = df["words_with_errors"] = None
allowed_chars_regex = "[^0-9A-ZÁÉÍÓÚÑa-záéíóúäëïöüñ\s\'\-]+"
differente_words_count_idxs = []

for _, row in df.iterrows():
    sentence = re.sub(allowed_chars_regex, '', row["sentence"])
    sentence_with_errors = re.sub(allowed_chars_regex, '', row["sentence_with_errors"])
    
    sentence_word_list = sentence.strip().split(' ')
    sentence_with_errors_word_list = sentence_with_errors.strip().split(' ')    
    
    if(len(sentence_word_list) != len(sentence_with_errors_word_list)):
        differente_words_count_idxs.append(row["number"])
        continue
    
    errors_count = 0
    words = []
    word_errors = []
    for idx, word in enumerate(sentence_word_list):        
        if word != sentence_with_errors_word_list[idx]:
            errors_count += 1
            words.append(word)
            word_errors.append(sentence_with_errors_word_list[idx])

    df.loc[df["number"] == row["number"], "sentence"] = sentence
    df.loc[df["number"] == row["number"], "sentence_with_errors"] = sentence_with_errors
    df.loc[df["number"] == row["number"], "words"] = ','.join(words)
    df.loc[df["number"] == row["number"], "words_with_errors"] = ','.join(word_errors)


In [13]:
print("{} filas en donde el número de palabras de la oración sin errores ortográficos no es igual al número de palabras de la oracion con errores ortográficos: {}".format(len(differente_words_count_idxs), differente_words_count_idxs))

0 filas en donde el número de palabras de la oración sin errores ortográficos no es igual al número de palabras de la oracion con errores ortográficos: []


Remover las filas en donde las oraciones no coinciden en número de palabras

In [14]:
df = df[~df["number"].isin(differente_words_count_idxs)]

In [15]:
print("Número de oraciones: {}".format(df.shape[0]))

Número de oraciones: 2082


Agregar la columna "error_types_count" al dataframe que indique el número de "error_types"

In [16]:
df["word_errors_count"] = df.apply(lambda row: len(row["words_with_errors"].split(',')) if row["words_with_errors"] else 0, axis=1)

Remover las filas en donde no tengan ninguna palabra con errores ortográficos.

In [17]:
df = df[~(df["word_errors_count"] == 0)]

In [18]:
print("Número de oraciones: {}".format(df.shape[0]))

Número de oraciones: 2078


In [19]:
df.head()

,number,word,sentence,sentence_with_errors,error_types,error_types_count,words,words_with_errors,word_errors_count
1,2,ptsepotanu,gimatu ptsepotanu koyayma girixanu pejnuruneko...,jimatu ptsepotanu koyayma jirixanu pegnuruneko...,"SIN,SIN,SIN",3,"gimatu,girixanu,pejnuruneko","jimatu,jirixanu,pegnuruneko",3
2,3,wnikatinitkalu,wnikatinitkalu rusanate nukojiru xani rumuwatanu,unikatinitkalu rusanate nukogiru xani rumuwatanu,"SIN,SIN",2,"wnikatinitkalu,nukojiru","unikatinitkalu,nukogiru",2
3,4,katmaklege,wuya katmaklege gipxakwu gixo shimane wkashich...,wuya katmakleje gipxakwu jixo shimane wkashich...,"SIN,SIN,SIN",3,"katmaklege,gixo","katmakleje,jixo",2
4,5,ruxpakna,ruxpakna pejnuruneko yine chinyawakpotuko gits...,ruxpakna pegnuruneko yine chinyawakpotuko jits...,"SIN,SIN",2,"pejnuruneko,gitsrukatna","pegnuruneko,jitsrukatna",2
5,6,giwchina,giwchina repomginwu wutsrukate nyegwaklo namta...,giwchina repomjinwu wutsrukate niegwaklo namta...,"SIN,SIN,FON",3,"repomginwu,nyegwaklo","repomjinwu,niegwaklo",2


In [20]:
print("Número de oraciones sin errores ortográficos y con errores ortográficos (cada uno): {}".format(df.shape[0]))

Número de oraciones sin errores ortográficos y con errores ortográficos (cada uno): 2078


In [21]:
sentences = df["sentence"].tolist()
corpus = ' '.join(sentences)
words = [w.strip() for w in corpus.split()]
word_count = Counter(words)
unique_words = list(word_count.keys())

print("Número de palabras únicas (oraciones sin errores ortográficos): {}".format(len(unique_words)))

Número de palabras únicas (oraciones sin errores ortográficos): 6131


In [22]:
chars = sorted(list(set(df["sentence"].sum())))
print("{} caracteres (oraciones sin errores ortográficos): {}".format(len(chars), chars))

24 caracteres (oraciones sin errores ortográficos): [' ', 'a', 'c', 'e', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'ñ']


In [23]:
sentences = df["sentence_with_errors"].tolist()
corpus = ' '.join(sentences)
words = [w.strip() for w in corpus.split()]
word_count = Counter(words)
unique_words = list(word_count.keys())

print("Número de palabras únicas (oraciones con errores ortográficos): {}".format(len(unique_words)))

Número de palabras únicas (oraciones con errores ortográficos): 6710


In [24]:
chars = sorted(list(set(df["sentence_with_errors"].sum())))
print("{} caracteres (oraciones con errores ortográficos): {}".format(len(chars), chars))

24 caracteres (oraciones con errores ortográficos): [' ', 'a', 'c', 'e', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'ñ']


Se hace un shuffle de las oraciones

In [25]:
df = df.sample(frac=1, random_state=8).reset_index(drop=True)
n_head = df.shape[0] // 2
n_tail = df.shape[0] - n_head

Crear corpus de oraciones sin errores ortograficos

In [26]:
with open(language + ".teacher_general.val.sentences.txt", "w") as f:
    lines = (df[:n_head]["sentence"] + '\n').tolist()
    f.writelines(lines)

with open(language + ".teacher_general.test.sentences.txt", "w") as f:
    lines = (df[-n_tail:]["sentence"] + '\n').tolist()
    f.writelines(lines)

Crear corpus paralelos de oraciones con errores ortograficos

In [27]:
with open(language + ".teacher_general.val.sentences.errors.txt", "w") as f:
    lines = (df[:n_head]["sentence_with_errors"] + '\n').tolist()
    f.writelines(lines)

with open(language + ".teacher_general.test.sentences.errors.txt", "w") as f:
    lines = (df[-n_tail:]["sentence_with_errors"] + '\n').tolist()
    f.writelines(lines)

Ahora, crearemos un corpus de oraciones sin errores ortograficos y un corpus paralelo de oraciones con una palabra con error ortográfico de un tipo

Eliminamos las oraciones donde el valor de la columna "error_types_count" sea diferente al valor de la columna "word_errors_count"

In [28]:
df = df[~(df["error_types_count"] != df["word_errors_count"])]

In [29]:
print("Número de oraciones: {}".format(df.shape[0]))

Número de oraciones: 1616


In [30]:
errors = df["error_types"].tolist()
errors = ','.join(errors)
errors = [w.strip() for w in errors.split(',')]
errors_count = Counter(errors)

print("Número de tipos de errores: {}".format(len(errors)))
print("Tipos de errores: {}".format(dict(errors_count)))

Número de tipos de errores: 5266
Tipos de errores: {'SIN': 3916, 'FON': 1347, 'NUM': 2, 'GEN': 1}


Creamos el corpus de las oraciones sin errores ortográficos y un corpus paralelo de oraciones con un solo error ortográfico

In [31]:
dict_sentences_with_errors = {}
dict_sentences = {}

for error_type in errors_count:
    dict_sentences_with_errors[error_type] = []
    dict_sentences[error_type] = []

In [32]:
for _, row in df.iterrows():
    sentence = row["sentence"]
    error_types = row["error_types"].split(',')

    words = row["words"].split(',')
    words_with_errors = row["words_with_errors"].split(',')
    for idx, error_type in enumerate(error_types):
        dict_sentences[error_type].append(sentence + '\n')
        sentence_with_error = sentence.replace(words[idx], words_with_errors[idx]) + '\n'
        dict_sentences_with_errors[error_type].append(sentence_with_error)

for error_type in errors_count:
    with open("{}.teacher_{}.test.sentences.errors.txt".format(language, error_type.lower()), "w") as f:        
        f.writelines(dict_sentences_with_errors[error_type])

    with open("{}.teacher_{}.test.sentences.txt".format(language, error_type.lower()), "w") as f:
        f.writelines(dict_sentences[error_type])